# Aggregation and Grouping

An essential piece of analysis of large data is efficient summarization: computing aggregations like ``sum()``, ``mean()``, ``median()``, ``min()``, and ``max()``, in which a single number gives insight into the nature of a potentially large dataset.
In this section, we'll explore aggregations in Pandas, from simple operations akin to what we've seen on NumPy arrays, to more sophisticated operations based on the concept of a ``groupby``.

For convenience, we'll use the same ``display`` magic function that we've seen in previous sections:

In [2]:
import numpy as np
import pandas as pd

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

## Planets Data

Here we will use the Planets dataset, available via the [Seaborn package](http://seaborn.pydata.org/) (see [Visualization With Seaborn](04.14-Visualization-With-Seaborn.ipynb)).
It gives information on planets that astronomers have discovered around other stars (known as *extrasolar planets* or *exoplanets* for short). It can be downloaded with a simple Seaborn command:

In [3]:
import seaborn as sns
planets = sns.load_dataset('planets')
planets.shape

(1035, 6)

In [4]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


In [5]:
planets["method"].unique()

array(['Radial Velocity', 'Imaging', 'Eclipse Timing Variations',
       'Transit', 'Astrometry', 'Transit Timing Variations',
       'Orbital Brightness Modulation', 'Microlensing', 'Pulsar Timing',
       'Pulsation Timing Variations'], dtype=object)

This has some details on the 1,000+ extrasolar planets discovered up to 2014.

## Simple Aggregation in Pandas

Earlier, we explored some of the data aggregations available for NumPy arrays (["Aggregations: Min, Max, and Everything In Between"](02.04-Computation-on-arrays-aggregates.ipynb)).
As with a one-dimensional NumPy array, for a Pandas ``Series`` the aggregates return a single value:

In [6]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.rand(5))
ser

0    0.374540
1    0.950714
2    0.731994
3    0.598658
4    0.156019
dtype: float64

In [7]:
ser.sum()

2.811925491708157

In [8]:
ser.mean()

0.5623850983416314

For a ``DataFrame``, by default the aggregates return results within each column:

In [9]:
df = pd.DataFrame({'A': rng.rand(5),
                   'B': rng.rand(5)})
df

,A,B
0,0.155995,0.020584
1,0.058084,0.969910
2,0.866176,0.832443
3,0.601115,0.212339
4,0.708073,0.181825


In [10]:
df.mean()

A    0.477888
B    0.443420
dtype: float64

By specifying the ``axis`` argument, you can instead aggregate within each row:

In [11]:
df.mean(axis='columns')

0    0.088290
1    0.513997
2    0.849309
3    0.406727
4    0.444949
dtype: float64

Pandas ``Series`` and ``DataFrame``s include all of the common aggregates mentioned in [Aggregations: Min, Max, and Everything In Between](02.04-Computation-on-arrays-aggregates.ipynb); in addition, there is a convenience method ``describe()`` that computes several common aggregates for each column and returns the result.
Let's use this on the Planets data, for now dropping rows with missing values:

In [12]:
planets.isnull()

,method,number,orbital_period,mass,distance,year
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,False,False,False
4,False,False,False,False,False,False
...,...,...,...,...,...,...
1030,False,False,False,True,False,False
1031,False,False,False,True,False,False
1032,False,False,False,True,False,False
1033,False,False,False,True,False,False


In [13]:
planets.isnull().sum()

method              0
number              0
orbital_period     43
mass              522
distance          227
year                0
dtype: int64

In [14]:
planets.describe()

,number,orbital_period,mass,distance,year
count,1035.000000,992.000000,513.000000,808.000000,1035.000000
mean,1.785507,2002.917596,2.638161,264.069282,2009.070531
std,1.240976,26014.728304,3.818617,733.116493,3.972567
min,1.000000,0.090706,0.003600,1.350000,1989.000000
25%,1.000000,5.442540,0.229000,32.560000,2007.000000
50%,1.000000,39.979500,1.260000,55.250000,2010.000000
75%,2.000000,526.005000,3.040000,178.500000,2012.000000
max,7.000000,730000.000000,25.000000,8500.000000,2014.000000


In [15]:
planets.dropna().describe()

,number,orbital_period,mass,distance,year
count,498.00000,498.000000,498.000000,498.000000,498.000000
mean,1.73494,835.778671,2.509320,52.068213,2007.377510
std,1.17572,1469.128259,3.636274,46.596041,4.167284
min,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,2.00000,999.600000,2.867500,59.332500,2011.000000
max,6.00000,17337.500000,25.000000,354.000000,2014.000000


This can be a useful way to begin understanding the overall properties of a dataset.
For example, we see in the ``year`` column that although exoplanets were discovered as far back as 1989, half of all known expolanets were not discovered until 2010 or after.
This is largely thanks to the *Kepler* mission, which is a space-based telescope specifically designed for finding eclipsing planets around other stars.

The following table summarizes some other built-in Pandas aggregations:

| Aggregation              | Description                     |
|--------------------------|---------------------------------|
| ``count()``              | Total number of items           |
| ``first()``, ``last()``  | First and last item             |
| ``mean()``, ``median()`` | Mean and median                 |
| ``min()``, ``max()``     | Minimum and maximum             |
| ``std()``, ``var()``     | Standard deviation and variance |
| ``mad()``                | Mean absolute deviation         |
| ``prod()``               | Product of all items            |
| ``sum()``                | Sum of all items                |

These are all methods of ``DataFrame`` and ``Series`` objects.

To go deeper into the data, however, simple aggregates are often not enough.
The next level of data summarization is the ``groupby`` operation, which allows you to quickly and efficiently compute aggregates on subsets of data.

## GroupBy: Split, Apply, Combine

Simple aggregations can give you a flavor of your dataset, but often we would prefer to aggregate conditionally on some label or index: this is implemented in the so-called ``groupby`` operation.
The name "group by" comes from a command in the SQL database language, but it is perhaps more illuminative to think of it in the terms first coined by Hadley Wickham of Rstats fame: *split, apply, combine*.

### Split, apply, combine

A canonical example of this split-apply-combine operation, where the "apply" is a summation aggregation.

This makes clear what the ``groupby`` accomplishes:

- The *split* step involves breaking up and grouping a ``DataFrame`` depending on the value of the specified key.
- The *apply* step involves computing some function, usually an aggregate, transformation, or filtering, within the individual groups.
- The *combine* step merges the results of these operations into an output array.

While this could certainly be done manually using some combination of the masking, aggregation, and merging commands covered earlier, an important realization is that *the intermediate splits do not need to be explicitly instantiated*. Rather, the ``GroupBy`` can (often) do this in a single pass over the data, updating the sum, mean, count, min, or other aggregate for each group along the way.
The power of the ``GroupBy`` is that it abstracts away these steps: the user need not think about *how* the computation is done under the hood, but rather thinks about the *operation as a whole*.

As a concrete example, let's take a look at using Pandas for the computation shown in this diagram.
We'll start by creating the input ``DataFrame``:

In [16]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data': range(6)}, columns=['key', 'data'])
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


In [17]:
df_A = df[df["key"] == "A"]
df_B = df[df["key"] == "B"]
df_C = df[df["key"] == "C"]
display("df_A", "df_B", "df_C")

,key,data
0,A,0
3,A,3
,key,data
1,B,1
4,B,4
,key,data
2,C,2
5,C,5


In [18]:
pd.DataFrame({"A":[df_A["data"].mean()], "B":[df_B["data"].mean()], "C":[df_C["data"].mean()]}).T

,0
A,1.5
B,2.5
C,3.5


The most basic split-apply-combine operation can be computed with the ``groupby()`` method of ``DataFrame``s, passing the name of the desired key column:

In [19]:
df.groupby('key')

Notice that what is returned is not a set of ``DataFrame``s, but a ``DataFrameGroupBy`` object.
This object is where the magic is: you can think of it as a special view of the ``DataFrame``, which is poised to dig into the groups but does no actual computation until the aggregation is applied.
This "lazy evaluation" approach means that common aggregates can be implemented very efficiently in a way that is almost transparent to the user.

To produce a result, we can apply an aggregate to this ``DataFrameGroupBy`` object, which will perform the appropriate apply/combine steps to produce the desired result:

In [20]:
df.groupby('key').sum()

,data
key,
A,3
B,5
C,7


The ``sum()`` method is just one possibility here; you can apply virtually any common Pandas or NumPy aggregation function, as well as virtually any valid ``DataFrame`` operation, as we will see in the following discussion.

### The GroupBy object

The ``GroupBy`` object is a very flexible abstraction.
In many ways, you can simply treat it as if it's a collection of ``DataFrame``s, and it does the difficult things under the hood. Let's see some examples using the Planets data.

Perhaps the most important operations made available by a ``GroupBy`` are *aggregate*, *filter*, *transform*, and *apply*.
We'll discuss each of these more fully in ["Aggregate, Filter, Transform, Apply"](#Aggregate,-Filter,-Transform,-Apply), but before that let's introduce some of the other functionality that can be used with the basic ``GroupBy`` operation.

#### Column indexing

The ``GroupBy`` object supports column indexing in the same way as the ``DataFrame``, and returns a modified ``GroupBy`` object.
For example:

In [21]:
planets

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300000,7.10,77.40,2006
1,Radial Velocity,1,874.774000,2.21,56.95,2008
2,Radial Velocity,1,763.000000,2.60,19.84,2011
3,Radial Velocity,1,326.030000,19.40,110.62,2007
4,Radial Velocity,1,516.220000,10.50,119.47,2009
...,...,...,...,...,...,...
1030,Transit,1,3.941507,NaN,172.00,2006
1031,Transit,1,2.615864,NaN,148.00,2007
1032,Transit,1,3.191524,NaN,174.00,2007
1033,Transit,1,4.125083,NaN,293.00,2008


In [22]:
planets['method'].nunique()

10

In [23]:
planets.groupby('method')

In [24]:
planets.groupby('method')['orbital_period']

Here we've selected a particular ``Series`` group from the original ``DataFrame`` group by reference to its column name.
As with the ``GroupBy`` object, no computation is done until we call some aggregate on the object:

In [25]:
planets.groupby('method')['orbital_period'].median()

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

This gives an idea of the general scale of orbital periods (in days) that each method is sensitive to.

#### Iteration over groups

The ``GroupBy`` object supports direct iteration over the groups, returning each group as a ``Series`` or ``DataFrame``:

In [26]:
planets[planets['method'] == 'Radial Velocity']

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.3000,7.10,77.40,2006
1,Radial Velocity,1,874.7740,2.21,56.95,2008
2,Radial Velocity,1,763.0000,2.60,19.84,2011
3,Radial Velocity,1,326.0300,19.40,110.62,2007
4,Radial Velocity,1,516.2200,10.50,119.47,2009
...,...,...,...,...,...,...
914,Radial Velocity,1,6.9580,0.34,NaN,2014
915,Radial Velocity,1,5.1180,0.40,NaN,2014
916,Radial Velocity,1,121.7100,1.54,NaN,2014
939,Radial Velocity,1,4.4264,NaN,NaN,2012


In [27]:
planets[planets['method'] == 'Astrometry']

,method,number,orbital_period,mass,distance,year
113,Astrometry,1,246.36,NaN,20.77,2013
537,Astrometry,1,1016.00,NaN,14.98,2010


In [28]:
planets[planets['method'] == 'Eclipse Timing Variations']

,method,number,orbital_period,mass,distance,year
32,Eclipse Timing Variations,1,10220.00,6.05,NaN,2009
37,Eclipse Timing Variations,2,5767.00,NaN,130.72,2008
38,Eclipse Timing Variations,2,3321.00,NaN,130.72,2008
39,Eclipse Timing Variations,2,5573.55,NaN,500.00,2010
40,Eclipse Timing Variations,2,2883.50,NaN,500.00,2010
41,Eclipse Timing Variations,1,2900.00,NaN,NaN,2011
42,Eclipse Timing Variations,1,4343.50,4.20,NaN,2012
43,Eclipse Timing Variations,2,5840.00,NaN,NaN,2011
44,Eclipse Timing Variations,2,1916.25,NaN,NaN,2011


In [29]:
for (method, group) in planets.groupby('method')["orbital_period"]:
    print("{:30s} shape={} median={:.2f} max={:.2f}".format(method, group.shape, group.median(), group.max()))

Astrometry                     shape=(2,) median=631.18 max=1016.00
Eclipse Timing Variations      shape=(9,) median=4343.50 max=10220.00
Imaging                        shape=(38,) median=27500.00 max=730000.00
Microlensing                   shape=(23,) median=3300.00 max=5100.00
Orbital Brightness Modulation  shape=(3,) median=0.34 max=1.54
Pulsar Timing                  shape=(5,) median=66.54 max=36525.00
Pulsation Timing Variations    shape=(1,) median=1170.00 max=1170.00
Radial Velocity                shape=(553,) median=360.20 max=17337.50
Transit                        shape=(397,) median=5.71 max=331.60
Transit Timing Variations      shape=(4,) median=57.01 max=160.00


This can be useful for doing certain things manually, though it is often much faster to use the built-in ``apply`` functionality, which we will discuss momentarily.

#### Dispatch methods

Through some Python class magic, any method not explicitly implemented by the ``GroupBy`` object will be passed through and called on the groups, whether they are ``DataFrame`` or ``Series`` objects.
For example, you can use the ``describe()`` method of ``DataFrame``s to perform a set of aggregations that describe each group in the data:

In [30]:
planets.groupby('method')['year'].describe()#.unstack()

,count,mean,std,min,25%,50%,75%,max
method,,,,,,,,
Astrometry,2.0,2011.500000,2.121320,2010.0,2010.75,2011.5,2012.25,2013.0
Eclipse Timing Variations,9.0,2010.000000,1.414214,2008.0,2009.00,2010.0,2011.00,2012.0
Imaging,38.0,2009.131579,2.781901,2004.0,2008.00,2009.0,2011.00,2013.0
Microlensing,23.0,2009.782609,2.859697,2004.0,2008.00,2010.0,2012.00,2013.0
Orbital Brightness Modulation,3.0,2011.666667,1.154701,2011.0,2011.00,2011.0,2012.00,2013.0
Pulsar Timing,5.0,1998.400000,8.384510,1992.0,1992.00,1994.0,2003.00,2011.0
Pulsation Timing Variations,1.0,2007.000000,NaN,2007.0,2007.00,2007.0,2007.00,2007.0
Radial Velocity,553.0,2007.518987,4.249052,1989.0,2005.00,2009.0,2011.00,2014.0
Transit,397.0,2011.236776,2.077867,2002.0,2010.00,2012.0,2013.00,2014.0


In [31]:
[df.describe() for method, df in planets.groupby('method')['year']]

[count       2.00000
 mean     2011.50000
 std         2.12132
 min      2010.00000
 25%      2010.75000
 50%      2011.50000
 75%      2012.25000
 max      2013.00000
 Name: year, dtype: float64,
 count       9.000000
 mean     2010.000000
 std         1.414214
 min      2008.000000
 25%      2009.000000
 50%      2010.000000
 75%      2011.000000
 max      2012.000000
 Name: year, dtype: float64,
 count      38.000000
 mean     2009.131579
 std         2.781901
 min      2004.000000
 25%      2008.000000
 50%      2009.000000
 75%      2011.000000
 max      2013.000000
 Name: year, dtype: float64,
 count      23.000000
 mean     2009.782609
 std         2.859697
 min      2004.000000
 25%      2008.000000
 50%      2010.000000
 75%      2012.000000
 max      2013.000000
 Name: year, dtype: float64,
 count       3.000000
 mean     2011.666667
 std         1.154701
 min      2011.000000
 25%      2011.000000
 50%      2011.000000
 75%      2012.000000
 max      2013.000000
 Name: year,

Looking at this table helps us to better understand the data: for example, the vast majority of planets have been discovered by the Radial Velocity and Transit methods, though the latter only became common (due to new, more accurate telescopes) in the last decade.
The newest methods seem to be Transit Timing Variation and Orbital Brightness Modulation, which were not used to discover a new planet until 2011.

This is just one example of the utility of dispatch methods.
Notice that they are applied *to each individual group*, and the results are then combined within ``GroupBy`` and returned.
Again, any valid ``DataFrame``/``Series`` method can be used on the corresponding ``GroupBy`` object, which allows for some very flexible and powerful operations!

### Aggregate, filter, transform, apply

The preceding discussion focused on aggregation for the combine operation, but there are more options available.
In particular, ``GroupBy`` objects have ``aggregate()``, ``filter()``, ``transform()``, and ``apply()`` methods that efficiently implement a variety of useful operations before combining the grouped data.

For the purpose of the following subsections, we'll use this ``DataFrame``:

In [32]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


#### Aggregation

We're now familiar with ``GroupBy`` aggregations with ``sum()``, ``median()``, and the like, but the ``aggregate()`` method allows for even more flexibility.
It can take a string, a function, or a list thereof, and compute all the aggregates at once.
Here is a quick example combining all these:

In [33]:
def fun(f):
    return f("Hola Mundo")

fun(print)

Hola Mundo


In [34]:
df_aux = df.groupby('key').aggregate(['min', np.median, max])
df_aux.columns = ['_'.join(tup) for tup in df_aux.columns]
df_aux

,data1_min,data1_median,data1_max,data2_min,data2_median,data2_max
key,,,,,,
A,0,1.5,3,3,4.0,5
B,1,2.5,4,0,3.5,7
C,2,3.5,5,3,6.0,9


In [35]:
['_'.join(tup) for tup in df_aux.columns]

['d_a_t_a_1___m_i_n',
 'd_a_t_a_1___m_e_d_i_a_n',
 'd_a_t_a_1___m_a_x',
 'd_a_t_a_2___m_i_n',
 'd_a_t_a_2___m_e_d_i_a_n',
 'd_a_t_a_2___m_a_x']

In [36]:
df.groupby('key')["data1"].aggregate(['min', np.median, max])

,min,median,max
key,,,
A,0,1.5,3
B,1,2.5,4
C,2,3.5,5


In [37]:
df.groupby('key')["data1"].agg(['min', np.median, max])

,min,median,max
key,,,
A,0,1.5,3
B,1,2.5,4
C,2,3.5,5


Another useful pattern is to pass a dictionary mapping column names to operations to be applied on that column:

In [38]:
df.groupby('key').aggregate({'data1': 'min',
                             'data2': 'max'})

,data1,data2
key,,
A,0,5
B,1,7
C,2,9


#### Filtering

A filtering operation allows you to drop data based on the group properties.
For example, we might want to keep all groups in which the standard deviation is larger than some critical value:

In [39]:
def filter_func(x):
    return x['data2'].mean() >= 4

display('df', "df.groupby('key').mean()", "df.groupby('key').filter(filter_func)")

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby('key').mean()
     data1  data2
key              
A      1.5    4.0
B      2.5    3.5
C      3.5    6.0

df.groupby('key').filter(filter_func)
  key  data1  data2
0   A      0      5
2   C      2      3
3   A      3      3
5   C      5      9

In [40]:
np.random.seed(0)
df_ninos = pd.DataFrame({'clase':[1, 1, 1, 2, 2, 2, 3, 3, 4],
                            'genero':['H', 'H', 'M', 'M', 'H', 'M', 'H', 'M', 'H'],
                            'altura':(np.random.random(9) * 0.5 + 1),
                            'peso':(np.random.random(9) * 10 + 25)})
df_ninos

,clase,genero,altura,peso
0,1,H,1.274407,28.834415
1,1,H,1.357595,32.917250
2,1,M,1.301382,30.288949
3,2,M,1.272442,30.680446
4,2,H,1.211827,34.255966
5,2,M,1.322947,25.710361
6,3,H,1.218794,25.871293
7,3,M,1.445887,25.202184
8,4,H,1.481831,33.326198


In [41]:
df_ninos.groupby('clase')['altura'].mean()

clase
1    1.311128
2    1.269072
3    1.332340
4    1.481831
Name: altura, dtype: float64

In [42]:
df_ninos.groupby(['clase', 'genero'])['altura'].mean()

clase  genero
1      H         1.316001
       M         1.301382
2      H         1.211827
       M         1.297694
3      H         1.218794
       M         1.445887
4      H         1.481831
Name: altura, dtype: float64

In [43]:
df_ninos.groupby(['clase', 'genero'])['altura'].aggregate(['mean', np.median, max, min, np.std, np.var])

mean    median       max       min       std       var
clase genero                                                            
1     H       1.316001  1.316001  1.357595  1.274407  0.058823  0.003460
      M       1.301382  1.301382  1.301382  1.301382       NaN       NaN
2     H       1.211827  1.211827  1.211827  1.211827       NaN       NaN
      M       1.297694  1.297694  1.322947  1.272442  0.035713  0.001275
3     H       1.218794  1.218794  1.218794  1.218794       NaN       NaN
      M       1.445887  1.445887  1.445887  1.445887       NaN       NaN
4     H       1.481831  1.481831  1.481831  1.481831       NaN       NaN

In [44]:
df_ninos.groupby('clase').aggregate({'altura':['mean'],
                                        'peso': [min, max]})

altura       peso           
           mean        min        max
clase                                
1      1.311128  28.834415  32.917250
2      1.269072  25.710361  34.255966
3      1.332340  25.202184  25.871293
4      1.481831  33.326198  33.326198

In [45]:
def filter_height(x):
    return x['altura'].mean() > 1.3

display("df_ninos.groupby('clase')[['altura']].mean()", "df_ninos.groupby('clase').filter(filter_height)")

df_ninos.groupby('clase')[['altura']].mean()
         altura
clase          
1      1.311128
2      1.269072
3      1.332340
4      1.481831

df_ninos.groupby('clase').filter(filter_height)
   clase genero    altura       peso
0      1      H  1.274407  28.834415
1      1      H  1.357595  32.917250
2      1      M  1.301382  30.288949
6      3      H  1.218794  25.871293
7      3      M  1.445887  25.202184
8      4      H  1.481831  33.326198

In [46]:
df_ninos.groupby('clase')['altura'].mean()

clase
1    1.311128
2    1.269072
3    1.332340
4    1.481831
Name: altura, dtype: float64

The filter function should return a Boolean value specifying whether the group passes the filtering. Here because group A does not have a standard deviation greater than 4, it is dropped from the result.

#### Transformation

While aggregation must return a reduced version of the data, transformation can return some transformed version of the full data to recombine.
For such a transformation, the output is the same shape as the input.
A common example is to center the data by subtracting the group-wise mean:

In [47]:
def linear(x):
    return 4*x + 5

linear(1)

9

In [48]:
linear = lambda x, y: 4*x + 3*y + 5
linear(1, 0)

9

In [49]:
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


In [50]:
df[df['key'] == "A"] - df[df['key'] == "A"].mean()

C:\Users\alber\AppData\Local\Temp\ipykernel_31232\2519372630.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df[df['key'] == "A"] - df[df['key'] == "A"].mean()


,data1,data2,key
0,-1.5,1.0,NaN
3,1.5,-1.0,NaN


In [51]:
df.groupby('key').transform(lambda x: x - x.mean())

,data1,data2
0,-1.5,1.0
1,-1.5,-3.5
2,-1.5,-3.0
3,1.5,-1.0
4,1.5,3.5
5,1.5,3.0


In [52]:
df_means = df.groupby("key").mean().reset_index()
df_means

,key,data1,data2
0,A,1.5,4.0
1,B,2.5,3.5
2,C,3.5,6.0


In [53]:
df_aux = df.merge(df_means, on="key", suffixes=["", "_mean"], how="left")
df_aux

,key,data1,data2,data1_mean,data2_mean
0,A,0,5,1.5,4.0
1,B,1,0,2.5,3.5
2,C,2,3,3.5,6.0
3,A,3,3,1.5,4.0
4,B,4,7,2.5,3.5
5,C,5,9,3.5,6.0


In [54]:
df_aux["data1_dif"] = df_aux["data1"] - df_aux["data1_mean"]
df_aux["data2_dif"] = df_aux["data2"] - df_aux["data2_mean"]
df_aux

,key,data1,data2,data1_mean,data2_mean,data1_dif,data2_dif
0,A,0,5,1.5,4.0,-1.5,1.0
1,B,1,0,2.5,3.5,-1.5,-3.5
2,C,2,3,3.5,6.0,-1.5,-3.0
3,A,3,3,1.5,4.0,1.5,-1.0
4,B,4,7,2.5,3.5,1.5,3.5
5,C,5,9,3.5,6.0,1.5,3.0


In [55]:
df_aux = df.merge(df_means, on="key", suffixes=["", "_mean"], how="left")
df_aux["data1_dif"] = df_aux["data1"] - df_aux["data1_mean"]
df_aux["data2_dif"] = df_aux["data2"] - df_aux["data2_mean"]
df_aux[["data1_dif", "data2_dif"]]

,data1_dif,data2_dif
0,-1.5,1.0
1,-1.5,-3.5
2,-1.5,-3.0
3,1.5,-1.0
4,1.5,3.5
5,1.5,3.0


#### The apply() method

The ``apply()`` method lets you apply an arbitrary function to the group results.
The function should take a ``DataFrame``, and return either a Pandas object (e.g., ``DataFrame``, ``Series``) or a scalar; the combine operation will be tailored to the type of output returned.

For example, here is an ``apply()`` that normalizes the first column by the sum of the second:

In [56]:
df_ninos

,clase,genero,altura,peso
0,1,H,1.274407,28.834415
1,1,H,1.357595,32.917250
2,1,M,1.301382,30.288949
3,2,M,1.272442,30.680446
4,2,H,1.211827,34.255966
5,2,M,1.322947,25.710361
6,3,H,1.218794,25.871293
7,3,M,1.445887,25.202184
8,4,H,1.481831,33.326198


In [57]:
def desv_avg(x):
    x['rel_val'] = x['altura'] / x['peso'].mean()
    x['dif_mean_alt'] = x['altura'] - x['altura'].mean()
    
    return x
df_ninos.groupby('clase', group_keys=False).apply(desv_avg)

,clase,genero,altura,peso,rel_val,dif_mean_alt
0,1,H,1.274407,28.834415,0.041538,-0.036721
1,1,H,1.357595,32.917250,0.044250,0.046467
2,1,M,1.301382,30.288949,0.042418,-0.009746
3,2,M,1.272442,30.680446,0.042112,0.003370
4,2,H,1.211827,34.255966,0.040106,-0.057245
5,2,M,1.322947,25.710361,0.043784,0.053875
6,3,H,1.218794,25.871293,0.047727,-0.113546
7,3,M,1.445887,25.202184,0.056620,0.113546
8,4,H,1.481831,33.326198,0.044464,0.000000


In [58]:
df_ninos[['altura', 'peso']].apply(lambda x: x - x[6], axis=0)

,altura,peso
0,0.055613,2.963122
1,0.138801,7.045957
2,0.082588,4.417656
3,0.053648,4.809153
4,-0.006966,8.384673
5,0.104153,-0.160932
6,0.000000,0.000000
7,0.227093,-0.669109
8,0.263038,7.454905


In [59]:
def norm_by_data2(x):
    # x is a DataFrame of group values
    x['data3'] = x["data1"] / x['data2'].sum()
    return x

display('df', "df.groupby('key', group_keys=False).apply(norm_by_data2)")

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby('key', group_keys=False).apply(norm_by_data2)
  key  data1  data2     data3
0   A      0      5  0.000000
1   B      1      0  0.142857
2   C      2      3  0.166667
3   A      3      3  0.375000
4   B      4      7  0.571429
5   C      5      9  0.416667

`apply` contiene a `transform`

In [60]:
def norm_by_data2(x):
    # x is a DataFrame of group values
    # x['data3'] = x["data1"] / x['data2'].sum()
    x = x - x.mean()
    return x

display('df', "df.groupby('key', group_keys=True)[['data1', 'data2']].apply(norm_by_data2)")

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby('key', group_keys=True)[['data1', 'data2']].apply(norm_by_data2)
       data1  data2
key                
A   0   -1.5    1.0
    3    1.5   -1.0
B   1   -1.5   -3.5
    4    1.5    3.5
C   2   -1.5   -3.0
    5    1.5    3.0

In [61]:
def fun(x):
    return x["data1"] + x["data2"]
df.apply(fun, axis=1)

0     5
1     1
2     5
3     6
4    11
5    14
dtype: int64

``apply()`` within a ``GroupBy`` is quite flexible: the only criterion is that the function takes a ``DataFrame`` and returns a Pandas object or scalar; what you do in the middle is up to you!

### Specifying the split key

In the simple examples presented before, we split the ``DataFrame`` on a single column name.
This is just one of many options by which the groups can be defined, and we'll go through some other options for group specification here.

#### A list, array, series, or index providing the grouping keys

The key can be any series or list with a length matching that of the ``DataFrame``. For example:

In [62]:
L = [0, 1, 0, 1, 2, 0]
display('df', 'df.groupby(L).sum()')

<string>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
<string>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.


df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby(L).sum()
   data1  data2
0      7     17
1      4      3
2      4      7

In [63]:
L = [0, 1, 0, 1, 2, 0]
df['key_alt'] = L
display('df', 'df.groupby("key_alt").sum()')

<string>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
<string>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.


df
  key  data1  data2  key_alt
0   A      0      5        0
1   B      1      0        1
2   C      2      3        0
3   A      3      3        1
4   B      4      7        2
5   C      5      9        0

df.groupby("key_alt").sum()
         data1  data2
key_alt              
0            7     17
1            4      3
2            4      7

Of course, this means there's another, more verbose way of accomplishing the ``df.groupby('key')`` from before:

In [64]:
display('df', "df.groupby(df['key']).sum()")

df
  key  data1  data2  key_alt
0   A      0      5        0
1   B      1      0        1
2   C      2      3        0
3   A      3      3        1
4   B      4      7        2
5   C      5      9        0

df.groupby(df['key']).sum()
     data1  data2  key_alt
key                       
A        3      8        1
B        5      7        3
C        7     12        0

#### A dictionary or series mapping index to group

Another method is to provide a dictionary that maps index values to the group keys:

In [65]:
df2 = df.set_index('key')
mapping = {'A': 'vowel', 'B': 'consonant', 'C': 'consonant'}
display('df2', 'df2.groupby(mapping).sum()')

,data1,data2,key_alt
key,,,
A,0,5,0
B,1,0,1
C,2,3,0
A,3,3,1
B,4,7,2
C,5,9,0
,data1,data2,key_alt
key,,,
consonant,12,19,3


In [66]:
df_animals = pd.DataFrame({"especie":["hombre", "perro", "perro", "gato"], 
                           "altura":[1.70, 0.50, 0.60, 0.25],
                           "peso":[70, 20, 23, 5]})
df_animals.index = df_animals["especie"]
df_animals.drop(columns="especie", inplace=True)
df_animals

,altura,peso
especie,,
hombre,1.70,70
perro,0.50,20
perro,0.60,23
gato,0.25,5


In [67]:
d_mapping = {"hombre":"humanos", "perro":"animales", "gato":"animales"}

df_animals.groupby(d_mapping).mean()

,altura,peso
especie,,
animales,0.45,16.0
humanos,1.70,70.0


In [68]:
df_animals_aux = df_animals.reset_index()
df_animals_aux

,especie,altura,peso
0,hombre,1.70,70
1,perro,0.50,20
2,perro,0.60,23
3,gato,0.25,5


In [69]:
df_animals_aux["tipo"] = df_animals_aux["especie"].map(d_mapping)
df_animals_aux

,especie,altura,peso,tipo
0,hombre,1.70,70,humanos
1,perro,0.50,20,animales
2,perro,0.60,23,animales
3,gato,0.25,5,animales


In [70]:
df_animals_aux.groupby("tipo")[["altura", "peso"]].mean()

,altura,peso
tipo,,
animales,0.45,16.0
humanos,1.70,70.0


In [71]:
df_animals_aux.groupby("tipo")[["altura", "peso"]].agg([np.min, np.max, np.median, np.mean])

altura                   peso                  
           amin amax median  mean amin amax median  mean
tipo                                                    
animales   0.25  0.6    0.5  0.45    5   23   20.0  16.0
humanos    1.70  1.7    1.7  1.70   70   70   70.0  70.0

#### Any Python function

Similar to mapping, you can pass any Python function that will input the index value and output the group:

In [72]:
display('df2', 'df2.groupby(level=0).mean()')

,data1,data2,key_alt
key,,,
A,0,5,0
B,1,0,1
C,2,3,0
A,3,3,1
B,4,7,2
C,5,9,0
,data1,data2,key_alt
key,,,
A,1.5,4.0,0.5


In [73]:
display('df2', 'df2.groupby(str.lower).mean()')

,data1,data2,key_alt
key,,,
A,0,5,0
B,1,0,1
C,2,3,0
A,3,3,1
B,4,7,2
C,5,9,0
,data1,data2,key_alt
key,,,
a,1.5,4.0,0.5


#### A list of valid keys

Further, any of the preceding key choices can be combined to group on a multi-index:

In [74]:
df2_multi = df2.groupby([str.lower, mapping]).mean()
df2_multi.index

MultiIndex([('a',     'vowel'),
            ('b', 'consonant'),
            ('c', 'consonant')],
           names=['key', 'key'])

In [75]:
df2_multi.reset_index(level=1)

,key,data1,data2,key_alt
key,,,,
a,vowel,1.5,4.0,0.5
b,consonant,2.5,3.5,1.5
c,consonant,3.5,6.0,0.0


### Grouping example

As an example of this, in a couple lines of Python code we can put all these together and count discovered planets by method and by decade:

In [76]:
planets

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300000,7.10,77.40,2006
1,Radial Velocity,1,874.774000,2.21,56.95,2008
2,Radial Velocity,1,763.000000,2.60,19.84,2011
3,Radial Velocity,1,326.030000,19.40,110.62,2007
4,Radial Velocity,1,516.220000,10.50,119.47,2009
...,...,...,...,...,...,...
1030,Transit,1,3.941507,NaN,172.00,2006
1031,Transit,1,2.615864,NaN,148.00,2007
1032,Transit,1,3.191524,NaN,174.00,2007
1033,Transit,1,4.125083,NaN,293.00,2008


In [77]:
decade = planets['year']//10 * 10
decade = decade.astype(str) + 's'
planets['decade'] = decade
planets

,method,number,orbital_period,mass,distance,year,decade
0,Radial Velocity,1,269.300000,7.10,77.40,2006,2000s
1,Radial Velocity,1,874.774000,2.21,56.95,2008,2000s
2,Radial Velocity,1,763.000000,2.60,19.84,2011,2010s
3,Radial Velocity,1,326.030000,19.40,110.62,2007,2000s
4,Radial Velocity,1,516.220000,10.50,119.47,2009,2000s
...,...,...,...,...,...,...,...
1030,Transit,1,3.941507,NaN,172.00,2006,2000s
1031,Transit,1,2.615864,NaN,148.00,2007,2000s
1032,Transit,1,3.191524,NaN,174.00,2007,2000s
1033,Transit,1,4.125083,NaN,293.00,2008,2000s


In [81]:
planets_sr = planets.groupby(['decade', 'method'])['method'].count()
planets_sr.name = 'method_count'
planets_sr.unstack(level=0, fill_value=0)

decade,1980s,1990s,2000s,2010s
method,,,,
Astrometry,0,0,0,2
Eclipse Timing Variations,0,0,3,6
Imaging,0,0,20,18
Microlensing,0,0,10,13
Orbital Brightness Modulation,0,0,0,3
Pulsar Timing,0,3,1,1
Pulsation Timing Variations,0,0,1,0
Radial Velocity,1,28,309,215
Transit,0,0,62,335


In [156]:
planets_sr.unstack()

method,Astrometry,Eclipse Timing Variations,Imaging,Microlensing,Orbital Brightness Modulation,Pulsar Timing,Pulsation Timing Variations,Radial Velocity,Transit,Transit Timing Variations
decade,,,,,,,,,,
1980s,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
1990s,NaN,NaN,NaN,NaN,NaN,3.0,NaN,28.0,NaN,NaN
2000s,NaN,3.0,20.0,10.0,NaN,1.0,1.0,309.0,62.0,NaN
2010s,2.0,6.0,18.0,13.0,3.0,1.0,NaN,215.0,335.0,4.0


In [151]:
# decade = 10 * (planets['year'] // 10)
# decade = decade.astype(str) + 's'
# decade.name = 'decade'
# df_plan = planets.groupby(['method', decade]).sum()#.unstack().fillna(0)
# df_plan

In [152]:
# df_plan.reset_index(level=0)

In [118]:
df_plan

decade,1980s,1990s,2000s,2010s
method,,,,
Astrometry,0.0,0.0,0.0,2.0
Eclipse Timing Variations,0.0,0.0,5.0,10.0
Imaging,0.0,0.0,29.0,21.0
Microlensing,0.0,0.0,12.0,15.0
Orbital Brightness Modulation,0.0,0.0,0.0,5.0
Pulsar Timing,0.0,9.0,1.0,1.0
Pulsation Timing Variations,0.0,0.0,1.0,0.0
Radial Velocity,1.0,52.0,475.0,424.0
Transit,0.0,0.0,64.0,712.0


This shows the power of combining many of the operations we've discussed up to this point when looking at realistic datasets.
We immediately gain a coarse understanding of when and how planets have been discovered over the past several decades!

Here I would suggest digging into these few lines of code, and evaluating the individual steps to make sure you understand exactly what they are doing to the result.
It's certainly a somewhat complicated example, but understanding these pieces will give you the means to similarly explore your own data.